In [ ]:
pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

In [ ]:
pip install "s3transfer<0.7.0,>=0.6.0" "botocore<1.22.9,>=1.22.8" --upgrade

In [ ]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True) # timeout

In [1]:
import os

try:
    import torch # module이 뭐가 필요한지 확인, 도커 requirement로 하는 방법도 있음
except ImportError:
    os.system('pip install torch==1.10.2')

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import IPython

#install_needed = True
install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U transformers s3fs datasets # dependencies
    IPython.Application.instance().kernel.do_shutdown(True)

In [2]:
# create session and define role

import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

In [ ]:
# Use pre-trained model

from transformers import pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

conversation = '''Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
Jeff: ok.
Jeff: and how can I get started? 
Jeff: where can I find documentation? 
Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face                                           
'''
summarizer(conversation)

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [5]:
# deploy

from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'philschmid/bart-large-cnn-samsum', # s3에 업로드 한 후 tar로 옮겨야 하나요?
    'HF_TASK':'summarization'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, # IAM role
)

# deploy model to SageMaker Inference, create endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.xlarge', # ec2 instance type
    wait=False
)

# request - s3 load (hard limit: 6MB), result: dynamoDB
# streaming(real-time)/batch(input,output-s3) -> batch 추론, 비동기
# 
predictor.predict({
    'inputs': "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
})

# batch
batch_job = huggingface_estimator.transformer(
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    strategy='SingleRecord')


batch_job.transform(
    data='s3://s3-uri-to-batch-data',
    content_type='application/json',    
    split_type='Line')


# boto3 3줄
# sdk -> 대근님 코드 참고, api 호출로 한번에 가능

--------!

[{'summary_text': 'The Eiffel Tower is 324 metres tall and the second tallest free-standing structure in France after the Millau Viaduct. It was the first structure to reach a height of 300 metres and it is now taller than the Chrysler Building by 5.2 metres.'}]

In [ ]:
# create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.local import LocalSession

endpoint_name = huggingface_model
local_sess = LocalSession()

model_predictor = Predictor(
    endpoint_name = endpoint_name,
    sagemaker_session = local_sess,
    serializer=CSVSerializer()
)

In [ ]:
# wait for creating endpoint
# (1) deploy가 진행되는 동안 다른 셀을 작동시키기 위함
# (2) deploy 과정이 주피터 랩이 아닌 AWS에서 진행된다는 것을 보여주기 위함
from IPython.core.display import display, HTML
def make_endpoint_link(region, endpoint_name, endpoint_task):
    endpoint_link = f'<b><a target="blank" href="https://console.aws.amazon.com"'
    return endpoint_link

endpoint_link = make_endpoint_link(region, model_predictor.endpoint_name, '[Dep]')
display(HTML(endpoint_link))

In [ ]:
sess.wait_for_endpoint(model_predictor.endpoint_name, poll=5)

In [ ]:
# delete endpoint
predictor.delete_endpoint()